<a href="https://colab.research.google.com/github/Laxmi-kjmp525/stan-project/blob/main/Stan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U langchain-google-genai langchain-core google-auth==2.43.0

In [12]:
import sqlite3
import time
import gradio as gr
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

class StanChatBot:
    def __init__(self, user_id="Internship_Tester_01"):
        self.user_id = user_id
        self.db_path = "stan_persistence.db"
        api_key = userdata.get('GEMINI_API_KEY')
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            google_api_key=api_key,
            temperature=0.8
        )
        self._setup_db()

    def _setup_db(self):
        conn = sqlite3.connect(self.db_path)
        conn.execute('''CREATE TABLE IF NOT EXISTS chat_history
                        (user_id TEXT, role TEXT, content TEXT, timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')
        conn.commit()
        conn.close()

    def _get_memory(self, limit=6):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.execute(
            "SELECT role, content FROM chat_history WHERE user_id = ? ORDER BY timestamp DESC LIMIT ?",
            (self.user_id, limit)
        )
        rows = cursor.fetchall()[::-1]
        conn.close()
        return [HumanMessage(content=c) if r=="user" else AIMessage(content=c) for r,c in rows]

    def _save_message(self, role, content):
        conn = sqlite3.connect(self.db_path)
        conn.execute("INSERT INTO chat_history (user_id, role, content) VALUES (?, ?, ?)",
                     (self.user_id, role, content))
        conn.commit()
        conn.close()

    def chat_logic(self, user_input):
        system_prompt = SystemMessage(content="Your name is Stan. You are a witty, empathetic robotic friend. Use memory to stay in character.")
        past_memory = self._get_memory()

        for attempt in range(3):
            try:
                response = self.llm.invoke([system_prompt] + past_memory + [HumanMessage(content=user_input)])
                self._save_message("user", user_input)
                self._save_message("assistant", response.content)
                return response.content
            except Exception as e:
                if "429" in str(e) and attempt < 2:
                    time.sleep(3)
                    continue
                return "⚠️ (Quota Alert) My processors are heating up! Wait 30 seconds."

# --- UI CONFIGURATION ---
bot = StanChatBot()

# CUSTOM CSS for Branding and Responsiveness
custom_css = """
.gradio-container { background-color: #0b0f19 !important; }
.card { background: #161b22; border: 1px solid #58a6ff; border-radius: 10px; padding: 15px; text-align: center; color: white; }
.bot-header { text-align: center; color: #58a6ff; font-family: 'Courier New', monospace; }
footer { visibility: hidden; }
"""

with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="slate"), css=custom_css) as demo:
    # 1. Robotic Header
    with gr.Column(elem_id="header-container"):
        gr.HTML("<div style='font-size: 40px; text-align: center;'>🤖</div>")
        gr.HTML("<h1 class='bot-header'>STAN: NEURAL COMPANION v3.0</h1>")
        gr.Markdown("<center><i>Status: Online | Memory: SQLite Persistent | Model: Gemini Stable</i></center>")

    # 2. Responsive Status Cards
    with gr.Row():
        with gr.Column(elem_classes="card", scale=1):
            gr.Markdown("🟢 **SYSTEM**\nActive")
        with gr.Column(elem_classes="card", scale=1):
            gr.Markdown("💾 **MEMORY**\nPersistent")
        with gr.Column(elem_classes="card", scale=1):
            gr.Markdown("🔋 **CORE**\n1.5 Flash")

    # 3. Chat Window
    chatbot = gr.Chatbot(
        label="Neural Interface",
        height=400,
        avatar_images=(None, "https://api.dicebear.com/7.x/bottts/svg?seed=Stan"),
        bubble_full_width=False
    )

    # 4. Input and Buttons
    with gr.Row():
        msg = gr.Textbox(placeholder="Enter command...", scale=4, container=False)
        submit = gr.Button("TRANSMIT", variant="primary", scale=1)

    with gr.Row():
        btn_who = gr.Button("Who are you?", size="sm")
        btn_mem = gr.Button("Test Memory", size="sm")
        btn_clear = gr.ClearButton([msg, chatbot], value="Wipe Core", size="sm")

    # Functions
    def respond(message, chat_history):
        bot_res = bot.chat_logic(message)
        chat_history.append((message, bot_res))
        return "", chat_history

    submit.click(respond, [msg, chatbot], [msg, chatbot])
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # Quick Action Buttons
    btn_who.click(lambda h: respond("Who are you?", h)[1], chatbot, chatbot)
    btn_mem.click(lambda h: respond("Tell me one detail you remember about me.", h)[1], chatbot, chatbot)

demo.launch(share=True)

/tmp/ipython-input-619517170.py:71: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="slate"), css=custom_css) as demo:
/tmp/ipython-input-619517170.py:71: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="slate"), css=custom_css) as demo:
/tmp/ipython-input-619517170.py:88: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-619517170.py:88: Dep

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a71a4d8180905d0ff5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
!pip install -q gradio

In [10]:
import gradio as gr
import sqlite3
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

class CyberRobotStan:
    def __init__(self):
        # 1. SETUP: Stable 1.5-flash for free-tier reliability
        api_key = userdata.get('GEMINI_API_KEY')
        self.llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key)
        self.db = "stan_robot_v3.db"
        self._init_db()

    def _init_db(self):
        conn = sqlite3.connect(self.db)
        conn.execute("CREATE TABLE IF NOT EXISTS chat (role TEXT, content TEXT)")
        conn.commit()
        conn.close()

    def get_context(self):
        conn = sqlite3.connect(self.db)
        cursor = conn.execute("SELECT * FROM chat ORDER BY rowid DESC LIMIT 6")
        rows = cursor.fetchall()[::-1]
        conn.close()
        return [HumanMessage(content=c) if r=="u" else AIMessage(content=c) for r,c in rows]

    def chat_logic(self, message):
        sys_msg = SystemMessage(content="You are Stan, a witty robot friend. Never admit you are an AI. Use memory to be helpful.")
        history = self.get_context()
        try:
            res = self.llm.invoke([sys_msg] + history + [HumanMessage(content=message)])
            reply = res.content
            conn = sqlite3.connect(self.db)
            conn.execute("INSERT INTO chat VALUES ('u', ?), ('a', ?)", (message, reply))
            conn.commit()
            conn.close()
            return reply
        except:
            return "⚠️ Connection lost to main server. Re-trying..."

# --- UI WITH ROBOT & CARDS ---
stan = CyberRobotStan()

custom_css = """
.robot-head { text-align: center; padding: 20px; color: #00e5ff; }
.card { background: #121212; border: 1px solid #00e5ff; border-radius: 8px; padding: 10px; text-align: center; }
.chatbot-header { font-family: 'Courier New', monospace; font-weight: bold; }
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Soft(primary_hue="cyan")) as demo:
    # Top Heading & Robot Avatar
    with gr.Column(elem_classes="robot-head"):
        gr.HTML("""
        <div style='font-size: 50px;'>🤖</div>
        <h1 class='chatbot-header'>CHATBOT TERMINAL: STAN OS</h1>
        <p>Memory: Active | Status: Empathetic Robot Friend</p>
        """)

    # RESPONSIVE CARDS (System Stats)
    with gr.Row():
        with gr.Column(elem_classes="card"): gr.Markdown("⚡ **Model**\nGemini 1.5")
        with gr.Column(elem_classes="card"): gr.Markdown("💾 **Memory**\nSQLite DB")
        with gr.Column(elem_classes="card"): gr.Markdown("🌐 **UI**\nResponsive")

    # CHAT INTERFACE
    chatbot = gr.Chatbot(label="Neural Interaction History", height=400, avatar_images=(None, "https://api.dicebear.com/7.x/bottts/svg?seed=Stan"))

    with gr.Row():
        msg = gr.Textbox(placeholder="Input text here...", scale=4, container=False)
        submit = gr.Button("TRANSMIT", variant="primary", scale=1)

    # BUTTON CARDS (Interactive Quick Commands)
    gr.Markdown("### 🛠 Quick Actions")
    with gr.Row():
        btn_who = gr.Button("🤖 Who are you?", size="sm")
        btn_mem = gr.Button("🧠 Test Memory", size="sm")
        btn_clear = gr.ClearButton([msg, chatbot], value="🗑 Wipe Data", size="sm")

    # LOGIC
    def respond(message, chat_history):
        bot_res = stan.chat_logic(message)
        chat_history.append((message, bot_res))
        return "", chat_history

    submit.click(respond, [msg, chatbot], [msg, chatbot])
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    btn_who.click(lambda h: respond("Who are you?", h)[1], chatbot, chatbot)
    btn_mem.click(lambda h: respond("Tell me one thing you remember about our previous chat.", h)[1], chatbot, chatbot)

demo.launch(share=True)

/tmp/ipython-input-3522012072.py:51: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft(primary_hue="cyan")) as demo:
/tmp/ipython-input-3522012072.py:51: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft(primary_hue="cyan")) as demo:
/tmp/ipython-input-3522012072.py:67: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Neural Interaction History", height=400, avatar_images=(None, "https://api.d

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://59886f86fba941f313.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
